# LangChain: Memory
While interacting with these models, the models usually dont store the previous conversational history. However, it is intuitive if the model keeps track of all the previous messages in order to maintain the context of the conversation.

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory
* ConversationEntityMemory

#### NOTE: Useful Links to discover other memory types
https://python.langchain.com/docs/modules/memory/types/

## 1. ConversationBufferMemory

In [30]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

import warnings
warnings.filterwarnings("ignore")

In [31]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [84]:
llm = ChatOpenAI(temperature = 0.0, model = llm_model)
memory = ConversationBufferMemory( input_key = "input")
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = False # Shows the internal operations when set to True
)

In [77]:
conversation.predict(input = "Hi, my name is Shivam")

"Hello Shivam, it's nice to meet you. My name is AI. How can I assist you today?"

In [78]:
conversation.predict(input = "What is 1+1")

'The answer to 1+1 is 2.'

In [79]:
conversation.predict(input = "What is my name again?")

'Your name is Shivam.'

In [80]:
print(memory.buffer)

Human: Hi, my name is Shivam
AI: Hello Shivam, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1
AI: The answer to 1+1 is 2.
Human: What is my name again?
AI: Your name is Shivam.


In [81]:
# The empty curle brackets denote few of the options that can be passed.
# If there are no options, we send empty curles.
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Shivam\nAI: Hello Shivam, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1\nAI: The answer to 1+1 is 2.\nHuman: What is my name again?\nAI: Your name is Shivam."}

In [39]:
memory = ConversationBufferMemory()

memory.save_context(
    {"input": "Hi"},
    {"output": "Whats up?"}
)

In [40]:
print(memory.buffer)

Human: Hi
AI: Whats up?


In [41]:
memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: Whats up?'}

In [42]:
memory.save_context({"input": "Not much, just hanging"},
                   {"output": "Cool"})

In [43]:
memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: Whats up?\nHuman: Not much, just hanging\nAI: Cool'}

## ConversationBufferWindowMemory

- The conversation is stored based on the window provided. Anything outside the provided window will be discarded.  
- Pretty useful in order to just remember the most recent conversation and avoiding indefinite increase in the context provided.

In [44]:
from langchain.memory import ConversationBufferWindowMemory

In [45]:
memory = ConversationBufferWindowMemory(k = 1)

In [46]:
memory.save_context({"input": "Hi"},
                   {"output": "Whats up?"})

memory.save_context({"input": "Not much. Just hanging"},
                   {"output": "Cool"})

In [47]:
memory.load_memory_variables({})

{'history': 'Human: Not much. Just hanging\nAI: Cool'}

In [48]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [49]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [50]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [51]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I don't have access to that information. Could you please tell me your name?"

In [52]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is my name?
AI: I'm sorry, I don't have access to that information. Could you please tell me your name?
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I still don't have access to that information. However, I can tell you that your IP address is 123.456.789.012 and you are currently located in New York City. Is there anything else I can assist you with?"

## ConversationTokenBufferMemory

The memory will limit the number of tokens. As a lot of LLM providers provide limits as per the tokens, this allows to make the calculations directly.


In [53]:
#!pip install tiktoken

In [54]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [55]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [57]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

Idea: Instead of limiting the memory based on fixed number of sentences or tokens, lets use an llm to write the summary of the conversation so far, and the summary would be the memory.

In [58]:
from langchain.memory import ConversationSummaryBufferMemory

In [59]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [61]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments."}

In [62]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [63]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.


"Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."

In [64]:
memory.load_memory_variables({})

{'history': "System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.\nAI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience."}

In [65]:
conversation.predict(input="When should I work on LangChain?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's schedule. The AI informs the human of a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The human asks what would be a good demo to show.
AI: Based on the customer's interest in AI developments, I would suggest showcasing our latest natural language processing capabilities. We could demonstrate how our AI can accurately understand and respond to complex language queries, and even provide personalized recommendations based on the user's preferences. Additionally, we could highlight our AI's 

'You have a block of time scheduled for LangChain work after the morning meeting with the product team. That should give you a few hours to make progress on the project. If you need more time, we can adjust your schedule accordingly.'

In [66]:
conversation.predict(input="At what time should I work on LangChain?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI discuss the day's schedule, which includes a morning meeting with the product team, time to work on the LangChain project, and a lunch meeting with a customer interested in the latest AI developments. The AI suggests showcasing their latest natural language processing capabilities during the demo, highlighting the AI's ability to understand and respond to complex language queries and provide personalized recommendations based on user preferences. The AI also emphasizes the AI's ability to learn and adapt over time, making it a valuable tool for businesses looking to improve their customer experience.
Human: When should I work on LangChain?
AI: 

'You have a block of time scheduled for LangChain work starting at 10:30 AM, after the morning meeting with the product team.'

## There are additional memory types that can be used in various applications:
- **Vector Data Memory**: Stores text (from conversation or elsewhere ) in a vector database and retrieves the most relevant blocks of text.
- **Entity Memories**: Using an LLM , it remembers details about specific entities.
- **Traditional memory**: Storing the conversation in a conventional database (such as key-value store or SQL).

**NOTE:** You can also use **multiple memories** at one time.
E.g., <u>Conversation memory + Entity memory</u> to recall Individuals.

## Conversation Memory + Entity Memory

In [194]:
from langchain.memory import ConversationEntityMemory, \
ConversationSummaryBufferMemory, CombinedMemory
from langchain.prompts import PromptTemplate

In [196]:
memory = ConversationEntityMemory(llm = llm, input_key="input", chat_history_key="chat_history_lines")

In [204]:
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from its context. If the AI does \
not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
{chat_history_lines}
Entities
{entities}
Human: {input}
AI:"""

In [198]:
PROMPT = PromptTemplate(
    input_variables = ["chat_history_lines", "input", "entities"],
    template = _DEFAULT_TEMPLATE
)

In [199]:
memory.memory_variables

['entities', 'chat_history_lines']

In [200]:
conversation = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)

In [208]:
conversation.predict(input = "Hi")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
Human: Hi
AI: Hello! How can I assist you today?
Human: What are you doing?
AI: I am currently processing and analyzing data to provide accurate and helpful responses to your inquiries. Is there anything specific you would like me to assist you with?
Human: My name is Shivam Naik
AI: Yes, I am aware of that, Shivam Naik. Is there anything else you would like me to assist you with?
Entities
{}
Human: Hi
AI:

> Finished chain.


'Hello! How can I assist you today?'

In [206]:
conversation.predict(input = "What are you doing?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
Human: What are you doing?
AI: I am currently processing and analyzing data to provide accurate and helpful responses to your inquiries.
Human: My name is Shivam Naik
AI: Nice to meet you, Shivam Naik! How can I assist you today?
Human: Hi
AI: Hello! How can I assist you today?
Entities
{}
Human: What are you doing?
AI:

> Finished chain.


'I am currently processing and analyzing data to provide accurate and helpful responses to your inquiries. Is there anything specific you would like me to assist you with?'

In [207]:
conversation.predict(input = "My name is Shivam Naik")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
Human: My name is Shivam Naik
AI: Nice to meet you, Shivam Naik! How can I assist you today?
Human: Hi
AI: Hello! How can I assist you today?
Human: What are you doing?
AI: I am currently processing and analyzing data to provide accurate and helpful responses to your inquiries. Is there anything specific you would like me to assist you with?
Entities
{'Shivam Naik': 'Shivam Naik is the name of the human interacting with the AI assistant.'}
Human: My name is Shivam Naik
AI:

> Finished chain.


'Yes, I am aware of that, Shivam Naik. Is there anything else you would like me to assist you with?'

In [209]:
conversation.predict(input = "How many people do you know?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
Human: What are you doing?
AI: I am currently processing and analyzing data to provide accurate and helpful responses to your inquiries. Is there anything specific you would like me to assist you with?
Human: My name is Shivam Naik
AI: Yes, I am aware of that, Shivam Naik. Is there anything else you would like me to assist you with?
Human: Hi
AI: Hello! How can I assist you today?
Entities
{}
Human: How many people do you know?
AI:

> Finished chain.


'As an AI, I have access to a vast amount of information and data, but I do not "know" people in the traditional sense. However, I can provide information on the number of registered users or accounts on various platforms or databases if that is what you are looking for.'

In [211]:
conversation.run(input = "Who am I?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
Human: Hi
AI: Hello! How can I assist you today?
Human: How many people do you know?
AI: As an AI, I have access to a vast amount of information and data, but I do not "know" people in the traditional sense. However, I can provide information on the number of registered users or accounts on various platforms or databases if that is what you are looking for.
Human: Who am I?
AI: You are Shivam Naik, as you previously mentioned.
Entities
{'Shivam Naik': 'Shivam Naik is the name of the human interacting with the AI assistant.'}
Human: Who am I?
AI:

> Finished chain.


'You are Shivam Naik, as you previously mentioned.'